## 模式匹配
摘抄自《scala程序设计（第2版）》————第 4 章 模式匹配

乍一看,模式匹配似乎与你喜欢的类 C 语言中的 case 语句很相似。因为在常见的类 C 语言 case 语句中,你只能按顺序匹配简单的数据类型和表达式。例如,“在 i 为 5 的情况下,打印一条消息;在 i 为 6 的情况下,退出程序”。

而在 Scala 的模式匹配中,可以使用类型、通配符、序列、正则表达式,甚至可以深入获取对象的状态。这种对象状态的获取遵循一定的协议,也就是对象内部状态的可见性由该类型的实现来控制,这使得我们能够轻易获取暴露的状态并应用于变量中。对象状态的获取往往被称为“提取”或“解构”。

模式匹配可以用在许多代码场景中。最常用于 match 语句中,之后我们会给出其他的用法。

### 简单匹配
首先,我们通过匹配 Boolean 值来模拟掷硬币:
```scala
// src/main/scala/progscala2/patternmatching/match-boolean.sc

val bools = Seq(true, false)

for (bool <- bools) {
  bool match {
    case true => println("Got heads")
    case false => println("Got tails")
  }
}
```

这看起来就像是 C 风格的 case 语句。为了试验一下,你可以尝试将第二个 case false 语句注释掉,再运行脚本。这时你会得到一个警告和一个错误消息:
```
<console>:12: warning: match may not be exhaustive.
It would fail on the following input: false
                bool match {
                ^
Got heads
scala.MatchError: false (of class java.lang.Boolean)
    at .<init>(<console>:11)
    at .<clinit>(<console>)
...
```
**由于序列类型存在两种可能的取值: true 或 false ,因此编译器警告 match 语句未能覆盖所有可能的输入值。当尝试去匹配一个没有 case 语句的值时,我们发现编译器抛出了MatchError。**

我得提一下,以上例子的另一种替代写法是旧式的 if 语句:
```scala
val bools = Seq(true, false)

for (bool <- bools) {
  val which = if (bool) "head" else "tails"
  println("Got " + which)
}
```

### match 中的值 、 变量和类型
接下来,我们来讨论几种 match 语句。以下的例子能匹配特定的某个值,也能匹配特定类型的所有值,同时展示了 default 语句的写法来匹配任意输入值。
```scala
// src/main/scala/progscala2/patternmatching/match-variable.sc

for {
  x <- Seq(1, 2, 2.7, "one", "two", 'four)                           // <1>
} {
  val str = x match {                                                // <2>
    case 1          => "int 1"                                       // <3>
    case i: Int     => "other int: "+i                               // <4>
    case d: Double  => "a double: "+x                                // <5>
    case "one"      => "string one"                                  // <6>
    case s: String  => "other string: "+s                            // <7>
    case unexpected => "unexpected value: " + unexpected             // <8>
  }
  println(str)                                                       // <9>
}
```
➊ 由于序列元素包含不同类型,因此序列的类型为 Seq[Any] 。  
➋ x 的类型为 Any 。  
➌ 如果 x 等于 1 则匹配。  
➍ 匹配除 1 外的其他任意整数值。将 x 的值安全地转为 Int ,并赋值给 i 。  
➎ 匹配所有 Double 类型, x 的值被赋给 Double 型变量 d 。  
➏ 匹配字符串“one”。  
➐ 匹配除“one”外的其他任意字符串, x 的值被赋给了 String 类型的变量 x 。  
➑ 匹配其他任意输入, x 的值被赋给 unexpected 这个变量。由于未给出任何类型说明,unexpected 的类型被推断为 Any ,起到了 default 语句的功能。  
➒ 打印返回的字符串。  

为了使代码直观一些,我将 => (“箭头”) 排成一列。以下是程序的输出:
```
int 1
other int: 2
a double 2.7
string one
other string: two
unexpected value: 'four
```

像所有表达式一样,match 语句也会返回一个值。在这里,所有的子句都返回字符串,因此整个子句的返回值类型为 String 。**编译器会推断所有 case 子句返回值类型的最近公共父类型(也称为最小公共上限)作为返回值类型。**

**由于 x 类型为 Any ,因此我们需要足够的子句来覆盖所有可能的输入值。(对比一下我们用来匹配 Boolean 值的第一个例子。)这就是我们需要“default 子句”(使用 unexpected )的原因。然而,编写偏函数时,我们不需要覆盖所有可能的类型,因为它们是被有意设计的。**

匹配是按顺序进行的,因此具体的子句应该出现在宽泛的子句之前。否则,具体的语句将不可能有机会被匹配上。所以,默认子句必须是最后一个子句。幸运的是,编译器能识别这种类型的错误。

由于舍入误差的存在,两个看似相等的值可能由于最后一位有效数字的不同而被判断为不相等,我没有在示例中使用匹配浮点数字面量的子句。

以下是对之前例子的简单变形:
```scala
// src/main/scala/progscala2/patternmatching/match-variable2.sc

for {
  x <- Seq(1, 2, 2.7, "one", "two", 'four)
} {
  val str = x match {
    case 1          => "int 1"
    case _: Int     => "other int: "+x
    case _: Double  => "a double: "+x
    case "one"      => "string one"
    case _: String  => "other string: "+x
    case _          => "unexpected value: " + x
  }
  println(str)
}
```
我们用占位符 _ 替换了变量 i 、 d 、 s 和 unexpected 。事实上我们并不需要这些类型的对应变量值,只需要产生字符串。所以,可以在所有子句中使用 x 。

**除了偏函数,所有的 match 语句都必须是完全覆盖所有输入的。当输入类型为 Any 时,在结尾用 case _ 或 case some_name 作为默认子句。**

编写 case 子句时,有一些规则和陷阱需要注意。在被匹配或提取的值中,编译器假定以大写字母开头的为类型名,以小写字母开头的为变量名。

以下示例中的这条规则可能会使你感到惊讶:
```scala
// src/main/scala/progscala2/patternmatching/match-surprise.sc

def checkY(y: Int) = {
  for {
    x <- Seq(99, 100, 101)
  } {
    val str = x match {
      case y => "found y!"
      case i: Int => "int: "+i
    }
    println(str)
  }
}

checkY(100)
```
在第一个 case 子句中,我们希望它能匹配上一个可以由我们来指定的值,而不是一个硬编码的值。所以,我们可能会希望第一个 case 子句在 x 等于 y 时成功匹配, y 的值为 100 。脚本执行时将产生以下输出:
```
int: 99
found y!
int: 101
```
以下是我们获得的实际输出:
```
<console>:12: warning: patterns after a variable pattern cannot match (SLS 8.1.1)
If you intended to match against parameter y of method checkY, you must use
backticks, like: case `y` =>
            case y => "found y!"
                 ^
<console>:13: warning: unreachable code due to variable pattern 'y' on line 12
            case i: Int => "int: "+i
                                  ^
<console>:13: warning: unreachable code
            case i: Int => "int: "+i
                                  ^
checkY: (y: Int)Unit
found y!
found y!
found y!
```
**case y 的含义其实是:匹配所有输入(由于这里没有类型注解),并将其赋值给新的变量 y 。这里的 y 没有被解释为方法参数 y 。**因此,事实上我们将一个默认的、匹配一切的语句写在了第一个,导致系统给出了这条“变量型匹配语句”会匹配一切输入的警告。我们的代码也从未执行到第二条 case 语句,于是就得到了两条关于不可达代码的警告。在这里 SLS 8.1.1指《Scala 语法规范》(http://www.scala-lang.org/docu/files/ScalaReference.pdf)的 8.1.1 节。

第一条错误信息已经告诉我们应该怎么做:**使用反引号表示真正想要匹配的是参数y的值。**
```scala
// src/main/scala/progscala2/patternmatching/match-surprise-fix.sc

def checkY(y: Int) = {
  for {
    x <- Seq(99, 100, 101)
  } {
    val str = x match {
      case `y` => "found y!"           // The only change: `y`
      case i: Int => "int: "+i
    }
    println(str)
  }
}
checkY(100)
```
这时,输出就符合我们的期望了。

**在 case 子句中,以小写字母开头的标识符被认为是用来提取待匹配值的新变量。如果需要引用之前已经定义的变量时,应使用反引号将其包围。与此相对,以大写字母开头的标识符被认为是类型名称。(不过后面正则表达式的匹配似乎有点特殊)**

有时不同的匹配子句需要使用相同的处理代码,此时,为了避免代码冗余,我们可以将相同处理代码重构为一个单独的方法。同时, **case 子句支持“或”逻辑,使用 | 方法即可:**
```scala
// src/main/scala/progscala2/patternmatching/match-variable3.sc

for {
  x <- Seq(1, 2, 2.7, "one", "two", 'four)
} {
  val str = x match {
    case _: Int | _: Double => "a number: "+x
    case "one"              => "string one"
    case _: String          => "other string: "+x
    case _                  => "unexpected value: " + x
  }
  println(str)
}
```
现在, Int 和 Double 类型的值都能匹配上第一个 case 子句了。

### 序列的匹配
Seq (表 示“ 序 列 ”) 是具体的集合类型的父类型,这些集合类型支持以确定顺序遍历其元素,如 List 和 Vector 。

我们来考察用模式匹配和递归方法遍历 Seq 的传统方法,顺便学习一些关于序列的基础知识。
```scala
// src/main/scala/progscala2/patternmatching/match-seq.sc

val nonEmptySeq    = Seq(1, 2, 3, 4, 5)                              // <1>
val emptySeq       = Seq.empty[Int]
val nonEmptyList   = List(1, 2, 3, 4, 5)                             // <2>
val emptyList      = Nil
val nonEmptyVector = Vector(1, 2, 3, 4, 5)                           // <3>
val emptyVector    = Vector.empty[Int]
val nonEmptyMap    = Map("one" -> 1, "two" -> 2, "three" -> 3)       // <4>
val emptyMap       = Map.empty[String,Int]

def seqToString[T](seq: Seq[T]): String = seq match {                // <5>
  case head +: tail => s"$head +: " + seqToString(tail)              // <6>
  case Nil => "Nil"                                                  // <7>
}

for (seq <- Seq(                                                     // <8>
    nonEmptySeq, emptySeq, nonEmptyList, emptyList, 
    nonEmptyVector, emptyVector, nonEmptyMap.toSeq, emptyMap.toSeq)) {
  println(seqToString(seq))
}
```
➊ 构造一个非空的 Seq[Int] (事实上返回了一个 List );然后用惯用方法构造了一个空的 Seq[Int] 。  
➋ 构造一个非空的 List[Int] ( Seq 的一个子类型);然后用 Scala 库的一个专用对象 Nil,表示任意类型的空 List 。  
➌ 构 造 一 个 非 空 的 Vector[Int] ( Seq 的一个子类型);然后构造了一个空的 Vector[Int] 。  
➍ 构 造 了 一 个 非 空 的 Map[String,Int] ,这不是 Seq 的子类型。在接下来的讨论中我们会涉及这一点。 Map[String,Int] 的键为 String 类型,值为 Int 类型。然后构造了一个空的 Map[String,Int] 。  
➎ 定义了一个递归方法,从 Seq[T] 中构造 String , T 为某种待定的类型。方法体是用来与输入的 Seq[T] 相匹配。  
➏ 这里存在两个互斥的 match 子句。第一个子句匹配非空的 Seq ,提取其头部(第一个元素)以及尾部(除头部以外,剩下的元素)。( Seq 有 head 和 tail 方法,但在这里,这两个标识符按 case 子句的惯例被解释为变量。)在 case 子句中,用提取的头部加上“ +: ”,以及尾部的字符串表示来构造一个字符串。尾部的字符串表示由调用seqToString 产生。  
➐ 另外一个 case 只可能是空 Seq 。我们用表示空 List 专用的对象 Nil 来匹配。注意, 任何 Seq 的尾部都可以认为是以一个相应类型的空 Seq ,事实上, List 就是这么实现的。  
➑ 将以上这些 Seq 作为元素放到另一个大 Seq 中(对其中的 Map 调用 toSeq ,将其转为键-值对组成的序列),然后遍历该序列,并打印各个序列调用 seqToString 返回的结果。  

以下为执行结果:
```
1 +: 2 +: 3 +: 4 +: 5
Nil
1 +: 2 +: 3 +: 4 +: 5
Nil
1 +: 2 +: 3 +: 4 +: 5
Nil
(one,1) +: (two,2) +:
Nil
```

Map 并不是 Seq 的子类型,因为 Map 不保证遍历的顺序是固定的。因此,我们调用 Map.toSeq 创建一个键 - 值元组的序列。由此得到的 Seq 键值对(pair)将会按照插入的顺序进行遍历。这种副作用仅限于小的 Map 的实现上,并不是针对所有 Map 的一般性保证。这里的空集合表明 seqToString 方法对空集也能正常工作。

这里有两种新的 case 子句。第一个子句中, head +: tail 匹配了序列的头部和尾部。 +:操作符是序列的“构造”操作符,与我们在优先规则中为List 使用的 :: 操作符类似。回想一下,以冒号 (:) 结尾的方法向右结合,即向 Seq 的尾部结合。

虽然它们被称作“操作符”和“方法”,但其实并不太准确;我们会在之后的章节讨论这些表达式,现在先来关注几个关键点。

首先, case 子句只匹配至少包含一个头部元素的非空序列,它将序列的头部和剩下的部分分别提取到可变变量 head 和 tail 中。

第二,重申一下,这里的 head 和 tail 是任意的两个变量名。然而, Seq 类型也分别存在两个名为 head 和 tail 的方法,用于提取序列的头部和尾部元素。通常情况下,我们可以从上下文中清晰地看出这两个标识符是函数还是变量。顺便提一下,对空序列调用这两个方法时,编译器会抛出异常。

Seq 的行为很符合链表的定义,因为在链表中,每个头结点除了含有自身的值以外,还指向链表的尾部(即链表剩下的元素),从而创建了一种层级结构,类似以下四个节点所组成的的序列。在这里尾部添加了一个空序列:(node1, (node2, (node3, (node4, (end))))

**Scala 库中有一个名为 Nil 的对象,可以匹配所有的空序列。我们甚至可以用 Nil 来表示非List 的其他空集合,因为序列对相等操作的实现都是一样的,不必精确匹配具体类型。**

以下是上述程序的一个变体,增加了括号,这次我们只使用了几个集合类型:
```scala
// src/main/scala/progscala2/patternmatching/match-seq-parens.sc

val nonEmptySeq    = Seq(1, 2, 3, 4, 5)
val emptySeq       = Seq.empty[Int]
val nonEmptyMap    = Map("one" -> 1, "two" -> 2, "three" -> 3)

def seqToString2[T](seq: Seq[T]): String = seq match {
  case head +: tail => s"($head +: ${seqToString2(tail)})"           // <1>
  case Nil => "(Nil)"
}

for (seq <- Seq(nonEmptySeq, emptySeq, nonEmptyMap.toSeq)) {
  println(seqToString2(seq))
}
```
➊ 重新格式化字符串,增加了外边的括号, (...) 。

如下所示,脚本输出清楚地显示了层级结构,每个“子列表”都被括号包围:
```
(1 +: (2 +: (3 +: (4 +: (5 +: (Nil))))))
(Nil)
((one,1) +: ((two,2) +: ((three,3) +: (Nil))))
```

我们只用了两个 case 子句和递归就处理了序列。这暗示了所有序列的基础特性:序列要么为空,要么非空。这听起来很老套,但一旦理解了这一点,你就会多一个基于“分治”法的工具。 processSeq 就多次使用该方法。

在 Scala 2.10 之前,处理 List 有另一种很相似的惯用方法:
```scala
// src/main/scala/progscala2/patternmatching/match-list.sc

val nonEmptyList = List(1, 2, 3, 4, 5)
val emptyList    = Nil

def listToString[T](list: List[T]): String = list match {
  case head :: tail => s"($head :: ${listToString(tail)})"           // <1>
  case Nil => "(Nil)"
}

for (l <- List(nonEmptyList, emptyList)) { println(listToString(l)) }
```
➊ 用 :: 代替 +: 。

输出也很类似:
```
(1 :: (2 :: (3 :: (4 :: (5 :: (Nil))))))
(Nil)
```

因此,使用 Seq 写代码更方便,因为它可以应用于所有子类型,包括 List 与 Vector 。

我们可以通过复制、粘贴上一个示例的输出,重新构造出原始的对象:
```scala
scala> val s1 = (1 +: (2 +: (3 +: (4 +: (5 +: Nil)))))
s1: List[Int] = List(1, 2, 3, 4, 5)

scala> val l = (1 :: (2 :: (3 :: (4 :: (5 :: Nil)))))
l: List[Int] = List(1, 2, 3, 4, 5)

scala> val s2 = (("one",1) +: (("two",2) +: (("three",3) +: Nil)))
s2: List[(String, Int)] = List((one,1), (two,2), (three,3), (four,4))

scala> val m = Map(s2 :_*)
m: scala.collection.immutable.Map[String,Int] =
Map(one -> 1, two -> 2, three -> 3, four -> 4)
```
值得注意, Map.apply 工厂方法需要一个可变参数列表,其中的参数是由 2 个元素组成的元组。所以,为了用序列 s2 来构造 Map ,我们只能用 :_* 惯用法来将序列 s2 转化为可变参数列表。

使用 +: 和 :: 时,构造方法与模式匹配(“解构”)有着优雅的对称关系。我们将在本章的后面部分探究它的实现方式,并提供可分析的示例。

### 元组的匹配
通过元组字面量,很容易对元组进行匹配:
```scala
// src/main/scala/progscala2/patternmatching/match-tuple.sc

val langs = Seq(
  ("Scala",   "Martin", "Odersky"),
  ("Clojure", "Rich",   "Hickey"),
  ("Lisp",    "John",   "McCarthy"))

for (tuple <- langs) {
  tuple match {
    case ("Scala", _, _) => println("Found Scala")                   // <1>
    case (lang, first, last) =>                                      // <2>
      println(s"Found other language: $lang ($first, $last)")
  }
}
```
➊ 匹配一个三元组的元组,其中第一个元素为字符串 Scala,忽略第二和第三个元素。  
➋ 匹配任意三元素元组,其中的元素可以为任意类型,但在这里,由于输入的值为lang , 元 素 类 型 被 推 断 为 String 。 元 组 的 三 个 元 素 被 提 取 到 变 量 lang 、 first 和last 中。  

该示例的输出如下:
```
Found Scala
Found other language: Clojure (Rich, Hickey)
Found other language: Lisp (John, McCarthy)
```

元素可以拆分为各个组成的元素。我们可以匹配元组中任意位置的字面量,同时忽略我们不需要的值。

### case 中的 guard 语句
在模式匹配中使用字面量的好处很多,但有时你却需要添加其他的逻辑:
```
// src/main/scala/progscala2/patternmatching/match-guard.sc

for (i <- Seq(1,2,3,4)) {
  i match {
    case _ if i%2 == 0 => println(s"even: $i")                       // <1>
    case _             => println(s"odd:  $i")                       // <2>
  }
}
```
➊ 只有当 i 为偶数时才匹配。我们用 _ 代替变量名,因为我们已经有 i 可以作为变量名了。  
➋ 匹配上一 case 子句相对的另一种可能性,即 i 为奇数。  

以上代码的输出为:
```
odd: 1
even: 2
odd: 3
even: 4
```

**注意, if 表达式的两边不需要使用括号,就像我们在 for 表达式中也不需要括号一样。**

### case 类的匹配
我们现在来看更多关于深度匹配的例子,并对 case 类对象的内容进行考察:
```scala
// src/main/scala/progscala2/patternmatching/match-deep.sc

// Simplistic address type. Using all strings is questionable, too.
case class Address(street: String, city: String, country: String)
case class Person(name: String, age: Int, address: Address)

val alice   = Person("Alice",   25, Address("1 Scala Lane", "Chicago", "USA"))
val bob     = Person("Bob",     29, Address("2 Java Ave.",  "Miami",   "USA"))
val charlie = Person("Charlie", 32, Address("3 Python Ct.", "Boston",  "USA"))

for (person <- Seq(alice, bob, charlie)) {
  person match {
    case Person("Alice", 25, Address(_, "Chicago", _)) => println("Hi Alice!")
    case Person("Bob", 29, Address("2 Java Ave.", "Miami", "USA")) => 
      println("Hi Bob!")
    case Person(name, age, _) => 
      println(s"Who are you, $age year-old person named $name?")
  }
}
```
输出如下:
```
Hi Alice!
Hi Bob!
Who are you, 32 year-old person named Charlie?
```

我 们 可 以 匹 配 嵌 套 类 型 的 内 容。 以 下 的 例 子 使 用 的 元 组 更 接 近 真 实 生 活。 想 象 一下,我们有一个 (String,Double) 元组组成的序列,表示商店里的商品名称与商品价格,同时想要将它们连同序号一起打印出来。为解决上面的问题,我们可以用到 Seq.zipWithIndex 方法:
```scala
// src/main/scala/progscala2/patternmatching/match-deep-tuple.sc

val itemsCosts = Seq(("Pencil", 0.52), ("Paper", 1.35), ("Notebook", 2.43))
val itemsCostsIndices = itemsCosts.zipWithIndex
for (itemCostIndex <- itemsCostsIndices) { 
  itemCostIndex match {
    case ((item, cost), index) => println(s"$index: $item costs $cost each")
  }
}
```
我们在 REPL 里运行以上脚本,用 :load 命令观察变量的类型和运行的输出(略加整理了
一下格式):
```scala
scala> :load src/main/scala/progscala2/patternmatching/match-deep-tuple.sc
Loading src/main/scala/progscala2/patternmatching/match-deep-tuple.sc...
itemsCosts: Seq[(String, Double)] =
 List((Pencil,0.52), (Paper,1.35), (Notebook,2.43))
itemsCostsIndices: Seq[((String, Double), Int)] =
 List(((Pencil,0.52),0), ((Paper,1.35),1), ((Notebook,2.43),2))
0: Pencil costs 0.52 each
1: Paper costs 1.35 each
2: Notebook costs 2.43 each
```
调用 zipWithIndex 时,返回的元组形式为 ((name,cost),index) 。通过匹配这种形式,我们提取了输入元组的三个元素,并将其打印出来。这样的代码是我经常使用的。

#### unapply 方法
除了 Scala 库里的类型以外,我们自定义的 case 类也可以使用类型匹配与提取的功能,它甚至还支持深度嵌套。

这是如何实现的呢?我们在 1.4 节已经了解到: case 类有一个伴随对象,伴随对象中有一个名为 apply 的工厂方法,用于构造对象。基于“对称”的观点,我们可以推断,一定还存在另一个名为 unapply 的自动生成的方法,用于提取和“解构”。事实上,确实存在这样的一个用于提取的方法,当遇见如下形式的类型匹配表达式时,该方法就会被调用:
```scala
person match {
  case Person("Alice", 25, Address(_, "Chicago", _)) => ...
  ...
}
```

Scala 找 到 Person.unapply(...) 和 Address.unapply(...) , 然 后 调 用 这 两 个 函 数。 所 有 的unapply 方法都返回 Option[TupleN[...]] ,此处的 N 表示可以从对象中提取的值的个数。在Person 这个 case 类中, N 为 3。被提取的值的类型与相应位置元组元素的类型一致。对于Person 而言,提取值的类型分别为 String 、 Int 和 Address 。所以,编译器生成的 Person的伴随对象是这样的:
```scala
object Person {
  def apply(name: String, age: Int, address: Address) =
    new Person(name, age, address)
  def unapply(p: Person): Option[Tuple3[String,Int,Address]] =
    Some((p.name, p.age, p.address))
  ...
}
```

既然编译器已经知道对象是 Person ,为什么 unapply 的返回值还要用 Option 呢? Scala 允许 unapply 方法“否决”这个匹配,返回 None ,这时,Scala 会使用下一个 case 子句。另外,如果我们不希望的话,可以不必暴露对象的所有属性。例如,如果我们不想暴露年龄隐私的话,可以选择不暴露 age 。我们会在 unapplySeq 方法中详细探讨这一细节,不过此时,只要知道被提取的属性以 Some 的形式返回即可,在本例中 Some 中的内容为 Tuple3 。编译器随后将元组 Tuple3 中的元素与字面值进行比较,比如匹配字符串 Alice;或者赋值给我们已经命名的变量;亦或者用 _ 占位符丢掉不需要的元素。

为了获得性能上的优势,Scala 2.11.1 放松了对 unapply 必须返回 Option[T]的要求。现在 unapply 能返回任意类型,只要该类型具有以下方法:
```scala
def isEmpty: Boolean
def get: T
```

如 果 有 必 要, unapply 方法会被递归调用。像本例中,我们在 Person 中有一个嵌套的Address 对象。类似地,在元组的那个例子中,我们也相应地递归调用了 unapply 方法。

**case 关键字被同时用于声明一种“特殊”的类,又用于 match 表达式中的 case 表达式,这可不是巧合。 case 类的特性就是为更便捷地进行模式匹配而设计的。**

在继续探索之前,注意一下,**返回类型 Option[Tuple3[String,Int,Address]] 的写法显得太过冗长。Scala 允许我们用元组字面语法来处理这种类型:**
```scala
val t1: Option[Tuple3[String,Int,Address]] = ...
val t2: Option[(String,Int,Address)] = ...
val t3: Option[ (String, Int, Address) ] = ...
```
**元组字面语法使得代码更易阅读;此外,逗号后的空格也有助于提高代码的可读性。**

现在让我们回到神秘的 head +: tail 表达式,去真正理解它的含义。我们可以看到 +: (构造)操作符可以通过在现有序列前追加新元素来构造新序列,我们可以这样凭空开始构造整个序列:
```scala
val list = 1 +: 2 +: 3 +: 4 +: Nil
```

由于 +: 是一个向右结合的操作符,因此我们首先将 4 追加到 Nil 中,再将 3 追加到产生的序列中,以此类推。

Scala 希望尽可能地支持构造和解构 / 提取的标准语法。这一点我们已经在序列、列表和元组中看到。另外,这些操作都是成对的,互为逆操作。

如果我们可以用一个名为 +: 的方法完成序列的构造,那么用相同的语法形式如何完成解构操作呢?刚才研究了 unapply 方法,但这是正确答案吗?虽然 Person.unapply 和 TupleN.unapply 知道在它们的实例中有多少“东西”,分别是 3 个和 N 个。但现在我们希望支持任意非空的集合。

为了完成这一点,Scala 库定义了一个特殊的单例对象,名为 +: 。是的,对象的名字为“ +: ”,类似于方法名,在 Scala 中,类型名可以使用的字符也很广泛。

这个类型只有一个方法,即编译器用来在 case 语句中进行提取操作的 unapply 方法。unapply 方法声明的示意就像是这样(我对实际声明稍微做了些简化,因为我们还没有涉及全部知识细节,不需要很多类型系统的知识来理解完整的方法签名):
```scala
def unapply[T, Coll](collection: Coll): Option[(T, Coll)]
```

头部的类型被推断为类型 T ;尾部被推断为某种集合类型 Coll 。 Coll 同时也是输入的集合类型。于是,方法返回了一个 Option ,其内容为输入集合的头部和尾部组成的两元素元组。

编译器如何才能在看到 case head +: tail => ... 表达式时调用 +:.unapply(collection)方法呢?我们需要把 case 子句写成 case +:(head, tail) => ... 才能与刚才示例中的模式匹配保持一致。

事实上,我们可以写成如下形式:
```scala
scala> def processSeq2[T](l: Seq[T]): Unit = l match {
  case +:(head, tail) =>
    printf("%s +: ", head)
    processSeq2(tail)
  case Nil => print("Nil")
}
scala> processSeq2(List(1,2,3,4,5))
1 +: 2 +: 3 +: 4 +: 5 +: Nil
```

我们也可以使用中缀表达式 head +: tail ,这是编译器的又一个语法糖。包含有两个类型参数的类型可以写为中缀表达式,同样, case 子句也可以这么做。考虑如下 REPL 中的代码:
```scala
// src/main/scala/progscala2/patternmatching/infix.sc

case class With[A,B](a: A, b: B)

// But notice the following type annotations:
val with1: With[String,Int] = With("Foo", 1)
val with2: String With Int  = With("Bar", 2)

List(with1, with2) foreach { w =>
  w match {
    case s With i => println(s"$s with $i")
    case _ => println(s"Unknown: $w")
  }
}
```

所以我们可以用两种形式书写类型签名: With[String,Int] 或者 String With Int 。后者更易阅读,不过缺乏经验的 Scala 程序员可能会感到困惑。请记住,尝试用类似的语法形式初始化一个值是不可行的:
```scala
scala> val w = "one" With 2
<console>:7: error: value With is not a member of String
        val w = "one" With 2
                      ^
```

List 也 有 这 样 的 一 个 类 似 的 对 象 —— :: 。如果你希望逆序处理序列中的每个元素时该怎么办呢?可以用 一个 对象进行处理! Scala 库的对象 :+ 可以让你匹配 List 的最后一个元素,然后从后往前依次访问各元素:
```scala
// src/main/scala/progscala2/patternmatching/match-reverse-seq.sc
// Compare to match-seq.sc

val nonEmptyList   = List(1, 2, 3, 4, 5)
val nonEmptyVector = Vector(1, 2, 3, 4, 5)
val nonEmptyMap    = Map("one" -> 1, "two" -> 2, "three" -> 3)

def reverseSeqToString[T](l: Seq[T]): String = l match {
  case prefix :+ end => reverseSeqToString(prefix) + s" :+ $end"
  case Nil => "Nil"
}

for (seq <- Seq(nonEmptyList, nonEmptyVector, nonEmptyMap.toSeq)) {
  println(reverseSeqToString(seq))
}
```
输出如下:
```
Nil :+ 1 :+ 2 :+ 3 :+ 4 :+ 5
Nil :+ 1 :+ 2 :+ 3 :+ 4 :+ 5
Nil :+ (one,1) :+ (two,2) :+ (three,3)
```

Nil 是第一个输出的元素,它的结合性是左结合的。同时,对于其他两个输入的 List 和 Vector ,也生成了相同的输出。

你应该比较一下 seqToString 和 reverseSeqToString 方法,因为它们各自使用不同的方式实现递归。在此之前,请确保你理解它们各自的工作机制。

像之前一样,你可以用输出的内容重新构造一个集合(第二行输出与第一行重复,可以忽略):
```scala
scala> Nil :+ 1 :+ 2 :+ 3 :+ 4 :+ 5
res0: List[Int] = List(1, 2, 3, 4, 5)
```

对于 List,用于追加元素的 :+ 方法以及用于模式匹配的 :+ 方法均需要 O(n)的时间复杂度,这两个方法都必须要从列表的头部遍历一遍。而对于其他某些序列,如 Vector,则需要 O(1) 的时间复杂度。

#### unapplySeq 方法
如 果 你 想 要 更 灵 活 一 些, 希 望 提 取 序 列 时 返 回 非 固 定 数 量 的 元 素, 该 怎 么 办 呢?unapplySeq 方 法 可 以 做 到 这 一 点。 除 了 apply 方 法 以 外, Seq 的 伴 随 对 象 还 实 现unapplySeq 方法, 而不是普通伴随对象的 unapply 方法:
```scala
def apply[A](elems: A*): Seq[A]
def unapplySeq[A](x: Seq[A]): Some[Seq[A]]
```

回顾一下,在这里 A* 表示 elems 是一个可变参数列表。下面的示例是对之前例子中 +: 的变形,这里我们将触发 unapplySeq 方法的调用,并考察所提取元素的“滑动窗口”:
```scala
// src/main/scala/progscala2/patternmatching/match-seq-unapplySeq.sc

val nonEmptyList   = List(1, 2, 3, 4, 5)                             // <1>
val emptyList      = Nil
val nonEmptyMap    = Map("one" -> 1, "two" -> 2, "three" -> 3)

// Process pairs
def windows[T](seq: Seq[T]): String = seq match {
  case Seq(head1, head2, _*) =>                                      // <2>
    s"($head1, $head2), " + windows(seq.tail)                        // <3>
  case Seq(head, _*) => 
    s"($head, _), " + windows(seq.tail)                              // <4>
  case Nil => "Nil"
}

for (seq <- Seq(nonEmptyList, emptyList, nonEmptyMap.toSeq)) {
  println(windows(seq))
}
```
➊ 定义了一个非空 List ,一个 Nil ,和一个 Map 。  
➋ 在 match 语句中,看起来我们似乎会隐含调用 Seq.apply(...) ,但实际上我们调用的是Seq.unapplySeq 。我们提取了前两个元素,忽略了用 _* 表示的其他可变参数。 * 表示另个或多个元素,与正则表达式中的 * 类似。  
➌ 用匹配到的前两个元没素格式化字符串,同时调用 seq.tail 将提取的“窗口”向后移动一位。注意,在这次匹配中,我们并有提取尾部元素。  
➍ 我们还需要匹配只有一个元素的序列,否则匹配就不完全。用 _ 表示不存在的“第二个”元素。我们已经知道调用 windows(seq.tail) 会返回 Nil ,但为了避免将字符串再重复一遍,我们再次调用了 windows 方法。  

滑动窗口的输出为:
```
(1, 2), (2, 3), (3, 4), (4, 5), (5, _), Nil
Nil
((one,1), (two,2)), ((two,2), (three,3)), ((three,3), _), Nil
```

我们依然可以在匹配中使用 +: ,这个写法更加优雅:
```scala
// src/main/scala/progscala2/patternmatching/match-seq-without-unapplySeq.sc

val nonEmptyList   = List(1, 2, 3, 4, 5)
val emptyList      = Nil
val nonEmptyMap    = Map("one" -> 1, "two" -> 2, "three" -> 3)

// Process pairs
def windows2[T](seq: Seq[T]): String = seq match {
  case head1 +: head2 +: tail => s"($head1, $head2), " + windows2(seq.tail)
  case head +: tail => s"($head, _), " + windows2(tail)
  case Nil => "Nil"
}

for (seq <- Seq(nonEmptyList, emptyList, nonEmptyMap.toSeq)) {
  println(windows2(seq))
}
```

滑动窗口如此有用, Seq 甚至提供了两个方法用于创建窗口:
```scala
scala> val seq = Seq(1,2,3,4,5)
seq: Seq[Int] = List(1, 2, 3, 4, 5)

scala> val slide2 = seq.sliding(2)
slide2: Iterator[Seq[Int]] = non-empty iterator

scala> slide2.toSeq
res0: Seq[Seq[Int]] = res56: Seq[Seq[Int]] = Stream(List(1, 2), ?)

scala> slide2.toList
res1: List[Seq[Int]] = List(List(1, 2), List(2, 3), List(3, 4), List(4, 5))

scala> seq.sliding(3,2).toList
res2: List[Seq[Int]] = List(List(1, 2, 3), List(3, 4, 5))
```

这两个 sliding 方法都返回迭代器,它们是“惰性”的。由于对大的序列进行复制代价太过昂贵,这两个函数都没有立即对所操作的列表进行复制。对返回的迭代器调用 toSeq 方法, 可 以 将 迭 代 器 转 为 一 个 collection.immutable.Stream 。这是一个惰性列表,创建时即对列表的头部元素求值,但只在需要的时候才对尾部元素求值。调用 toList 不一样,它返回一个 List ,创建时就求出了所有元素的值。

注意,输出的结果与之前的例子有一点点不同,例如,在这里输出的结尾没有 (5, _) 。

### 可变参数列表的匹配
在 2.6 节,我们介绍了 Scala 对方法中的可变参数列表的支持。例如,在编写一个与 SQL 交互的工具,或是用一个 case 类来表示 WHERE foo IN (val1, val2, ...) 这样的 SQL 语句(这个例子来自于真实项目而非开源代码的启发)时,我们用到了 Scala 对可变参数列表的支持。以下就是一个带可变参数列表的 case 类,用于处理 SQL 语句中的各个值。代码中还包括了另外一些定义,用于处理 WHERE x OP y 这样的 SQL 语句, OP 是 SQL 的比较操作符。
```scala
// src/main/scala/progscala2/patternmatching/match-vararglist.sc

// Operators for WHERE clauses
object Op extends Enumeration {                                      // <1>
  type Op = Value

  val EQ   = Value("=")
  val NE   = Value("!=")
  val LTGT = Value("<>")
  val LT   = Value("<")
  val LE   = Value("<=")
  val GT   = Value(">")
  val GE   = Value(">=")
}
import Op._

// Represent a SQL "WHERE x op value" clause, where +op+ is a 
// comparison operator: =, !=, <>, <, <=, >, or >=.
// 表示 SQL的 "WHERE x op value"语句,其中+op+为一个比较
// 操作符: =, !=, <>, <, <=, >, or >=。
case class WhereOp[T](columnName: String, op: Op, value: T)          // <2>

// Represent a SQL "WHERE x IN (a, b, c, ...)" clause.
// 表示SQL的"WHERE x IN (a, b, c, ...)" 语句。
case class WhereIn[T](columnName: String, val1: T, vals: T*)         // <3>

val wheres = Seq(                                                    // <4>
  WhereIn("state", "IL", "CA", "VA"),
  WhereOp("state", EQ, "IL"),
  WhereOp("name", EQ, "Buck Trends"),
  WhereOp("age", GT, 29))

for (where <- wheres) {
  where match {
    case WhereIn(col, val1, vals @ _*) =>                            // <5>
      val valStr = (val1 +: vals).mkString(", ")
      println (s"WHERE $col IN ($valStr)")
    case WhereOp(col, op, value) => println (s"WHERE $col $op $value")
    case _ => println (s"ERROR: Unknown expression: $where")
  }
}
```
➊ 定义了一个 Enumeration 用于表示比较 SQL 操作符,每个操作符有一个“名字”,是一个字符串。  
➋ 用于表示 WHERE x OP y 的 case 类。  
➌ 用于表示 WHERE x IN (val1, val2, ...) 的 case 类。  
➍ 用于解析的示例对象。  
➎ 注意匹配可变参数的语法形式 : name @ _* 。  

**用于匹配可变参数列表的语法 name @ _* 并不太符合直觉,但在有些场合你的确需要它。**以下是示例运行的输出:
```
WHERE state IN (IL, CA, VA)
WHERE state = IL
WHERE name = Buck Trends
WHERE age > 29
```

### 正则表达式的匹配
正则表达式可以很方便地从符合特定结构的字符串中提取数据。

Scala 封装了 Java 的正则表达式。以下给出一个示例:
```scala
// src/main/scala/progscala2/patternmatching/match-regex.sc

val BookExtractorRE = """Book: title=([^,]+),\s+author=(.+)""".r     // <1>
val MagazineExtractorRE = """Magazine: title=([^,]+),\s+issue=(.+)""".r

val catalog = Seq(
  "Book: title=Programming Scala Second Edition, author=Dean Wampler",
  "Magazine: title=The New Yorker, issue=January 2014",
  "Unknown: text=Who put this here??"
)

for (item <- catalog) {
  item match {
    case BookExtractorRE(title, author) =>                           // <2>
      println(s"""Book "$title", written by $author""")
    case MagazineExtractorRE(title, issue) =>
      println(s"""Magazine "$title", issue $issue""")
    case entry => println(s"Unrecognized entry: $entry")
  }
}
```
➊ 该正则表达式匹配一个用于表示书本的字符串,其中有两个捕捉组(注意正则表达式中的括号),一个表示标题,一个表示作者。调用 r 方法以创建正则表达式。第二个正则表达式匹配一个用于表示杂志的字符串,其中的捕捉组表示杂志标题和发行时间。  
➋ 用法与 case 类相似,与捕捉组相匹配的字符串被提取出来,赋值给变量。  

运行的输出为:
```
Book "Programming Scala Second Edition", written by Dean Wampler
Magazine "The New Yorker", issue January 2014
Unrecognized entry: Unknown: text=Who put this here??
```

`我们用三重双引号来表示正则表达式字符串,否则,就不得不对正则表达式的反斜杠进行转义,例如用 \\s 表示 \s 。你还可以通过创建一个 Regex 类的实例来定义正则表达式,如 new Regex("""\W""") ,但这种用法并不常见。`

在三个双引号内的正则表达式中使用变量插值是无效的。你依然需要对变量插值进行转义,例如,你应该用 `s"""$first\\s+$second""".r`,而不是`s"""$first\s+$second""".r`。而如果你没有使用变量插值,则不必转义。

scala.util.matching.Regex 定义了若干个用于正则表达式其他操作的方法,如查找和替换。

### 再谈 case 语句的变量绑定
假设以下场景:你**需要从对象中提取值,但你又想将一个变量赋给该对象的整体**。该怎么做呢?

我们来对前文中匹配 Person 类的属性的实例做以下修改。
```scala
// src/main/scala/progscala2/patternmatching/match-deep2.sc

case class Address(street: String, city: String, country: String)
case class Person(name: String, age: Int, address: Address)

val alice   = Person("Alice",   25, Address("1 Scala Lane", "Chicago", "USA"))
val bob     = Person("Bob",     29, Address("2 Java Ave.",  "Miami",   "USA"))
val charlie = Person("Charlie", 32, Address("3 Python Ct.", "Boston",  "USA"))

for (person <- Seq(alice, bob, charlie)) {
  person match {
    case p @ Person("Alice", 25, address) => println(s"Hi Alice! $p")
    case p @ Person("Bob", 29, a @ Address(street, city, country)) => 
      println(s"Hi ${p.name}! age ${p.age}, in ${a.city}")
    case p @ Person(name, age, _) => 
      println(s"Who are you, $age year-old person named $name? $p")
  }
}
```

p @ ... 的语法将整个 Person 类的实例赋值给了变量 p ,类似地, a @ ... 也将整个 Address 实例赋值给了变量。以下为修改版本的实例输出。(为适应排版做了格式化。)
```
Hi Alice! Person(Alice,25,Address(1 Scala Lane,Chicago,USA))
Hi Bob! age 29, in Miami
Who are you, 32 year-old person named Charlie? Person(Charlie,32,
  Address(3 Python Ct.,Boston,USA))
```

记住,**如果不需要从 Person 实例中提取属性值,我们只要写为 p: Person => ... 就可以了。**

### 再谈类型匹配
考虑以下例子,我们试图将输入的 List[Double] 和 List[String] 区分开:
```scala

// src/main/scala/progscala2/patternmatching/match-types.sc

for {
  x <- Seq(List(5.5,5.6,5.7), List("a", "b")) 
} yield (x match {
  case seqd: Seq[Double] => ("seq double", seqd)
  case seqs: Seq[String] => ("seq string", seqs)
  case _                 => ("unknown!", x)
})
```
```scala
<console>:11: warning: non-variable type argument Double in type pattern Seq[Double] (the underlying of Seq[Double]) is unchecked since it is eliminated by erasure
                case seqd: Seq[Double] => ("seq double", seqd)
                           ^
<console>:12: warning: non-variable type argument String in type pattern Seq[String] (the underlying of Seq[String]) is unchecked since it is eliminated by erasure
                case seqs: Seq[String] => ("seq string", seqs)
                           ^
<console>:12: warning: unreachable code
                case seqs: Seq[String] => ("seq string", seqs)
                                          ^
res0: Seq[(String, List[Any])] = List((seq double,List(5.5, 5.6, 5.7)), (seq double,List(a, b)))

```
**这些警告表示什么? Scala 运行于 JVM 中,这些警告来源于 JVM 的类型擦除,类型擦除是 Java 5 引入泛型后的一个历史遗留。为了避免与旧版本代码断代,JVM 的字节码不会记住一个泛型实例(如 List )中实际传入的类型参数的信息。**

所以,当编译器只能识别输入对象为 List ,但无法在运行时识别它是 List[Double] 还是List[String] 时,编译器就会发出警告。事实上,编译器认为第二个匹配 List[String] 的case 子句是不可达代码,意味着第一个匹配 List[Double] 的 case 子句可以匹配任意 List 。输出显示,对于两个输入,都打印出了 seq double。

一个不太美观但却有效的解决方法是:首先匹配集合,然后用一个嵌套的匹配语句去匹配集合中的第一个元素,从而决定其类型。这样的话,我们也就必须单独处理空序列:
```scala
// src/main/scala/progscala2/patternmatching/match-types2.sc

def doSeqMatch[T](seq: Seq[T]): String = seq match {
  case Nil => "Nothing"
  case head +: _ => head match {
    case _ : Double => "Double"
    case _ : String => "String"
    case _ => "Unmatched seq element"
  }
}

for {
  x <- Seq(List(5.5,5.6,5.7), List("a", "b"), Nil) 
} yield {
  x match {
    case seq: Seq[_] => (s"seq ${doSeqMatch(seq)}", seq)
    case _           => ("unknown!", x)
  }
}
```
以上脚本返回了期望的输出: Seq((seq Double,List(5.5, 5.6, 5.7)), (seq String,List(a,b)), (seq Nothing,List())) 。

### 封闭继承层级与全覆盖匹配
这一小节我们来讨论全覆盖匹配的需求和封闭类层级的应用场景。其实,我们在 2.10 节已经对封闭类做了介绍。我们用在以下代码中使用封闭继承层级类来表示 HTTP 协议的合法消息类型(或称为“方法”)。
```scala
// src/main/scala/progscala2/patternmatching/http.sc

sealed abstract class HttpMethod() {                                 // <1>
    def body: String                                                 // <2>
    def bodyLength = body.length
}

case class Connect(body: String) extends HttpMethod                  // <3>
case class Delete (body: String) extends HttpMethod
case class Get    (body: String) extends HttpMethod
case class Head   (body: String) extends HttpMethod
case class Options(body: String) extends HttpMethod
case class Post   (body: String) extends HttpMethod
case class Put    (body: String) extends HttpMethod
case class Trace  (body: String) extends HttpMethod

def handle (method: HttpMethod) = method match {                     // <4>
  case Connect (body) => s"connect: (length: ${method.bodyLength}) $body"
  case Delete  (body) => s"delete:  (length: ${method.bodyLength}) $body"
  case Get     (body) => s"get:     (length: ${method.bodyLength}) $body"
  case Head    (body) => s"head:    (length: ${method.bodyLength}) $body"
  case Options (body) => s"options: (length: ${method.bodyLength}) $body"
  case Post    (body) => s"post:    (length: ${method.bodyLength}) $body"
  case Put     (body) => s"put:     (length: ${method.bodyLength}) $body"
  case Trace   (body) => s"trace:   (length: ${method.bodyLength}) $body"
}

val methods = Seq(
  Connect("connect body..."),
  Delete ("delete body..."),
  Get    ("get body..."),
  Head   ("head body..."),
  Options("options body..."),
  Post   ("post body..."),
  Put    ("put body..."),
  Trace  ("trace body..."))

methods foreach (method => println(handle(method)))
```
➊ 定义了一个封闭的抽象基类。由于该类被定义为封闭的,其子类型必须定义在本文件内。  
➋ 为 HTTP 报文的消息体定义了一个方法。  
➌ 定义了 8 个继承自 HttpMethod 的 case 类,每个类均在构造方法中声明了参数 body:String 。由于每个类均为 case 类,因此该参数是一个 val ,它实现了 HttpMethod 的抽象方法 def 。  
➍ 这是一个全覆盖的模式匹配表达式。即使我们没有默认的匹配子句,也可以达到全覆盖,因为输入的参数 method 只可能是我们定义的 8 个 case 类的实例。  

**对封闭基类的实例做模式匹配时,如果 case 语句覆盖了所有当前文件定义的类型,那么匹配就是全覆盖的。由于不允许有其他用于自定义的子类型,随着项目演进,匹配的全覆盖性也不会丧失。**

由此可以得出的一个推论:如果类型的继承层级可能发生变化,就应该避免使用 sealed 。这取决于你原有的面向对象继承规则,包括多态方法的设计情况。如果你去掉 HttpMethod的 sealed 关键字,然后在本文件或其他文件新定义一个子类型,会怎么样呢?你必须在代码库以及客户端的代码库中找出并修改所有关于 HttpMethod 实例的模式匹配代码。

另外,这里还给出了一种实现某些方法的技巧。一个抽象的,不带参数的父类方法,在子类型中可以用一个 val 实现。这是由于 val 的值是固定的(必定的),而一个不带参数、返回值为某类型变量的方法可以返回任意一个该类型的变量。这样,使用 val 实现的方法在返回值上严格符合方法定义,当方法被“调用”时,使用 val 变量与真实调用方法一样安全。事实上,这是透明引用的一个应用。在透明引用中,我们用一个值代替一个总是返回固定值的表达式!

**在父类型中,不带参数的抽象方法可以在子类中用 val 变量实现。推荐的做法是:在抽象父类型中声明一个不带参数的抽象方法,这样就给子类型如何具体实现该方法留下了巨大的自由,既可以用方法实现,也可以用 val 变量实现。**

执行该脚本,产生以下输出:
```
connect: (length: 15) connect body...
delete:  (length: 14) delete body...
get:     (length: 11) get body...
head:    (length: 12) head body...
options: (length: 15) options body...
post:    (length: 12) post body...
put:     (length: 11) put body...
trace:   (length: 13) trace body...
```

HttpMethod 的 case 类很小,理论上我们也可以用 Enumeration 代替。但那样会有一个很大的缺陷,就是编译器就无法判断 Enumeration 相应的 match 语句是否全覆盖。如果我们在这里使用了 Enumeration ,而在 match 语句中忘记了匹配 Trace 的语句,那我们也只能在运行时抛出 MatchError 的时候才知道这个错误的存在。

**当使用类型匹配时避免使用枚举类型。编译器无法判断匹配语句是否全覆盖。**

### 模式匹配的其他用法
**幸运的是,模式匹配这一强大特性并不仅仅局限于 case 语句。在定义变量时也可以运用模式匹配,包括 for 表达式中的变量定义。**
```scala
scala> case class Address(street: String, city: String, country: String)
defined class Address

scala> case class Person(name: String, age: Int, address: Address)
defined class Person

scala> val Person(name, age, Address(_, state, _)) = Person("Dean", 29, Address("1 Scala Way", "CA", "USA"))
name: String = Dean
age: Int = 29
state: String = CA
```
没错,只用了一个步骤,我们就将 Person 中所有需要的属性抽取了出来,同时略过了不需要的属性。这个方法也可以用在 List 上。
```scala
scala> val head +: tail = List(1,2,3)
head: Int = 1
tail: List[Int] = List(2, 3)

scala> val head1 +: head2 +: tail = Vector(1,2,3)
head1: Int = 1
head2: Int = 2
tail: scala.collection.immutable.Vector[Int] = Vector(3)

scala>
a: Int
b: Int
c: Int
val Seq(a,b,c) = List(1,2,3)
= 1
= 2
= 3

scala> val Seq(a,b,c) = List(1,2,3,4)
scala.MatchError: List(1, 2, 3, 4) (of class collection.immutable.$colon$colon)
  ... 43 elided
```
这个技巧非常好用。在你自己的示例中尝试运用下吧。

在 if 表达式中我们也可以用模式匹配:
```scala
scala> val p = Person("Dean", 29, Address("1 Scala Way", "CA", "USA"))
p: Person = Person(Dean,29,Address(1 Scala Way,CA,USA))

scala> if (p == Person("Dean", 29, Address("1 Scala Way", "CA", "USA"))) "yes" else "no"
res0: String = yes

scala> if (p == Person("Dean", 29, Address("1 Scala Way", "CA", "USSR"))) "yes" else "no"
res1: String = no
```
然而,在这里无法使用 _ 占位符(必须不能啊，这里用的根本就不是模式匹配，就是普通的等号值比较):
```scala
scala> if (p == Person(_, 29, Address(_, _, "USA"))) "yes" else "no"
<console>:13: error: missing parameter type for expanded function ((x$1) => p.$eq$eq(Person(x$1, 29, ((x$2, x$3) => Address(x$2, x$3, "USA")))))
              if (p == Person(_, 29, Address(_, _, "USA"))) "yes" else "no"
  ...
```
`名为 $eq$eq 的内部函数用于 == 测试。因为在 JVM 规范中,只允许字母、数字、 _ 和 $ 作为标识符,Scala 对一些非字母数字的字符做了“字符映射”,使得它们符合 JVM 规范。在这里, = 变成了 $eq 。所有的映射规则会在第 22 章的表 22-1 中列出。`

**假设我们有一个函数,参数为整数序列,将所有整数的和与整数的个数放在元组中返回:**
```scala
scala> def sum_count(ints: Seq[Int]) = (ints.sum, ints.size)

scala> val (sum, count) = sum_count(List(1,2,3,4,5))
sum: Int = 15
count: Int = 5
```

这个用法我经常使用。在 3.6.5 节有一个人为生造的例子,它**在 for 表达式中使用模式匹配**。以下给出了该例子中与模式匹配相关的片段:
```scala
// src/main/scala/progscala2/patternmatching/scoped-option-for.sc

val dogBreeds = Seq(Some("Doberman"), None, Some("Yorkshire Terrier"), 
                    Some("Dachshund"), None, Some("Scottish Terrier"),
                    None, Some("Great Dane"), Some("Portuguese Water Dog"))

println("second pass:")
for {
  Some(breed) <- dogBreeds
  upcasedBreed = breed.toUpperCase()
} println(upcasedBreed)
```
如同前文,脚本输出依然为:
```
DOBERMAN
YORKSHIRE TERRIER
DACHSHUND
SCOTTISH TERRIER
GREAT DANE
PORTUGUESE WATER DOG
```

**模式匹配与 case 语句的另一个便利用法是,它们可以使带复杂参数的函数字面量更易于使用(就是使用偏函数):**
```scala
// src/main/scala/progscala2/patternmatching/match-fun-args.sc

case class Address(street: String, city: String, country: String)
case class Person(name: String, age: Int)

val as = Seq(
  Address("1 Scala Lane", "Anytown", "USA"),
  Address("2 Clojure Lane", "Othertown", "USA"))
val ps = Seq(
  Person("Buck Trends", 29),
  Person("Clo Jure", 28))

val pas = ps zip as

// Ugly way:
// 不太美观的方法:
pas map { tup =>
  val Person(name, age) = tup._1
  val Address(street, city, country) = tup._2
  s"$name (age: $age) lives at $street, $city, in $country"
}

// Nicer way:
// 不错的方法:
pas map {
  case (Person(name, age), Address(street, city, country)) =>
    s"$name (age: $age) lives at $street, $city, in $country"
}
```
注意,压缩序列的类型为 Seq[(Person,Address)] ,所以,我们传递给 map 的参数必须是一个类型为 (Person,Address) => String 的函数。我们给出了两个函数,第一个是一个“常规”函数,带一个元组参数,使用模式匹配从元组的两个元素中提取属性值。

第二个函数是一个偏函数,这在偏函数中有介绍。这种写法在语法上更为简洁,特别是需要从元组和更复杂的结构中抽取值时。但是需要记住:由于给出的是一个偏函数,所以case 表达式必须精确匹配输入,否则在运行时会抛出一个 MatchError 。

使用以上两个函数,产生的字符串序列均为:
```
List(
  "Buck Trends (age: 29) lives at 1 Scala Lane, Anytown, in USA",
  "Clo Jure (age: 28) lives at 2 Clojure Lane, Othertown, in USA")
```

**最后要介绍的是,我们可以在正则表达式中用模式匹配去解构字符串。**我写过一份用于SQL 解析的简单程序,以下是从该程序的测试代码中抽取的例子:
```scala
// src/main/scala/progscala2/patternmatching/regex-assignments.sc

val cols = """\*|[\w, ]+"""  // 用于提取列
val table = """\w+"""        // 用于提取表
val others = """.*"""
val selectRE = s"""SELECT\\s*(DISTINCT)?\\s+($cols)\\s*FROM\\s+($table)\\s*($others)?;""".r  // 用于其他语句

val selectRE(distinct1, cols1, table1, otherClauses1) = "SELECT DISTINCT * FROM atable;"
val selectRE(distinct2, cols2, table2, otherClauses2) = "SELECT col1, col2 FROM atable;"
val selectRE(distinct3, cols3, table3, otherClauses3) = "SELECT DISTINCT col1, col2 FROM atable;"
val selectRE(distinct4, cols4, table4, otherClauses4) = "SELECT DISTINCT col1, col2 FROM atable WHERE col1 = 'foo';"
```
注意,由于用了变量插值,因此在正则表达式字符串中,必须增加反斜杠进行转义,如用`\\s` 代替 `\s` 。

显然,用正则表达式去解析复杂文本如 XML 或编程语言,有其局限性。抛开简单的例子,我们考虑一个解析库就会明白正则表达式的局限性。我们会在第 20 章进一步讨论。

### 总结关于模式匹配的评价
**模式匹配是一个强大的“协议”,用于从数据结构中提取数据。**JavaBeans 模型有一个无意中造成的结果,就是模式匹配将会鼓励开发者用 getter 和 setter 暴露对象的属性。而这种做法往往忽略了一点,即状态应该被封装,只在恰当的时候才暴露出来,尤其对可变的属性而言更是如此。对状态信息的获取应该小心设计,以反映暴露的抽象。

考虑一下,当你需要以一种可控的方式抽取信息时,如何使用模式匹配?正如我们在unapply 方法中看到的那样,你可以自定义 unapply 方法去控制暴露出来的状态。这些方法允许你抽取信息的同时隐藏实现细节。实际上, unapply 方法返回的信息可能是类型的属性值经过转换后的结果。

**最后要说明的是:当设计模式匹配语句时,需要谨慎对待默认的 case 子句。在什么情况下,才应该出现“以上均不匹配”的情况呢?默认 case 子句有可能表明,你该改善一下程序的设计了。这样,你会更准确地知道程序中可能发生的所有匹配的情况。**

在 for 循环中,模式匹配的惯用方法使得 Scala 代码简单却又强大。对于功能相同的代码,Scala 程序的行数比 Java 程序少 10 倍的情况并不罕见。

所以,尽管 Java 8 增加匿名函数(Lambda)——一种类似模式匹配和 for 循环的工具——是一个巨大的改进,但 Scala 可以缩减几行代码的优势,是一个让你由 Java 转向 Scala 的理由。